# Project 4. Identify which knowledge-based techniques are appropriate for which task

### 1. Expert Systems:
- Project: Medical Diagnosis System
- Description: Build an expert system that takes patient symptoms as input and provides potential medical conditions as output.
- Techniques: Rule-based reasoning, forward chaining, inference engine.

To create an expert system for the Medical Diagnosis System, we'll use the pyknow library, which is a Python implementation of the CLIPS expert system shell. We'll define rules that infer potential medical conditions based on the patient's symptoms. Make sure you have the pyknow library installed before running the code. You can install it using pip install pyknow.

In [2]:
!pip install experta

In [3]:
from experta import *


# Define the Facts (Patient Symptoms)
class Symptom(Fact):
    pass

# Define the Medical Conditions
class MedicalCondition(Fact):
    pass


# Define the Expert System Rules
class MedicalDiagnosisExpertSystem(KnowledgeEngine):
    @Rule(Symptom(cough=True) & Symptom(fever=True) & Symptom(fatigue=True))
    def rule_flu(self):
        self.declare(MedicalCondition(name="Flu"))

    @Rule(Symptom(headache=True) & Symptom(stiff_neck=True) & Symptom(fever=True))
    def rule_meningitis(self):
        self.declare(MedicalCondition(name="Meningitis"))

    @Rule(Symptom(rash=True) & Symptom(fever=True) & Symptom(joint_pain=True))
    def rule_dengue(self):
        self.declare(MedicalCondition(name="Dengue"))

    @Rule(Symptom(chest_pain=True) & Symptom(shortness_of_breath=True))
    def rule_heart_attack(self):
        self.declare(MedicalCondition(name="Heart Attack"))


# Function to run the Medical Diagnosis Expert System
def run_medical_diagnosis_system(symptoms):
    engine = MedicalDiagnosisExpertSystem()

    for symptom, value in symptoms.items():
        engine.declare(Symptom(**{symptom: value}))

    engine.run()

    # Get the output medical condition
    condition = engine.facts.get(MedicalCondition)
    if condition:
        return condition['name']
    else:
        return "No specific medical condition found."


# Example Usage
if __name__ == "__main__":
    # Example symptoms (You can modify this dictionary with patient symptoms)
    patient_symptoms = {
        'cough': True,
        'fever': True,
        'fatigue': True,
    }

    medical_condition = run_medical_diagnosis_system(patient_symptoms)
    print("Medical Condition:", medical_condition)


Medical Condition: No specific medical condition found.
